In [15]:
import gc
import time

import torch

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('device:', device)
if torch.cuda.is_available():
    print('device name:', torch.cuda.get_device_name(device))

def collect(*, verbose=True):
    if verbose:
        print('garbage collector collected %d objects' % gc.collect())
    else:
        gc.collect()
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
        torch.cuda.reset_peak_memory_stats()

collect()

device: cuda
device name: NVIDIA GeForce RTX 3050 Laptop GPU
garbage collector collected 1934 objects


In [12]:
from IPython.display import display, Markdown

display(Markdown('Hello, **World**!'))

Hello, **World**!

In [2]:
from transformers import (
    MBartForConditionalGeneration,
    MBart50TokenizerFast,
)

model_name = 'facebook/mbart-large-50-many-to-many-mmt'

mbart_tokenizer = MBart50TokenizerFast.from_pretrained(model_name, device=device)
mbart_model = MBartForConditionalGeneration.from_pretrained(model_name)
mbart_model.to(device)
collect()

garbage collector collected 85 objects


In [26]:
def translate_article(article, source_lang, target_lang, *, verbose=False, device=device):
    global mbart_tokenizer, mbart_model
    mbart_tokenizer.src_lang = source_lang
    encoded = mbart_tokenizer(article, return_tensors='pt')
    encoded.to(device)
    if verbose:
        print('Tokenized article:')
        for key, value in encoded.items():
            print('%s: %s, %s, %s' % (key, str(value.size()), str(value.dtype), str(value.device)))
    tokens = mbart_model.generate(
        **encoded,
        forced_bos_token_id=mbart_tokenizer.lang_code_to_id[target_lang],
    )
    translated_article = mbart_tokenizer.batch_decode(tokens, skip_special_tokens=True)
    return translated_article[0]

In [30]:
article = '''
Đồng hồ sắp đánh tám giờ. Học trò tấp nập đi học, lũ năm lũ ba, tay cắp sách vừa đi vừa chuyện trò vui vẻ.

Đến trường, ai nấy vào học. Các lớp học đều rộng rãi, mát mẻ.

Thầy giáo hết lòng dạy các cậu, mà các cậu học hành rất chăm chỉ.

Sự học hành cần lắm. Ta phải rủ nhau đi học. Có học mới khôn được.
'''

now = time.time_ns()
with torch.no_grad():
    translated_article = translate_article(article, 'vi_VN', 'en_XX', verbose=True)
delta = time.time_ns() - now
print('Calculation duration: %.4f seconds' % (delta / (10 ** 9)))

collect()

display(Markdown(
'''
----
**⚜️ Original article**

%s

----
**⚜️ Translated article**

%s
''' % (article, translated_article)
))

Tokenized article:
input_ids: torch.Size([1, 97]), torch.int64, cuda:0
attention_mask: torch.Size([1, 97]), torch.int64, cuda:0
Calculation duration: 2.8569 seconds
garbage collector collected 11 objects



----
**⚜️ Original article**


Đồng hồ sắp đánh tám giờ. Học trò tấp nập đi học, lũ năm lũ ba, tay cắp sách vừa đi vừa chuyện trò vui vẻ.

Đến trường, ai nấy vào học. Các lớp học đều rộng rãi, mát mẻ.

Thầy giáo hết lòng dạy các cậu, mà các cậu học hành rất chăm chỉ.

Sự học hành cần lắm. Ta phải rủ nhau đi học. Có học mới khôn được.


----
**⚜️ Translated article**

The clock is eight o'clock. Students are rushing to school, fifth-graders, third-graders, they're grabbing books and they're walking and they're having a nice conversation. They're coming to school, everybody's eager to come. The classrooms are big, they're cool. The teacher's really teaching you, and you're learning hard. You've got to talk to each other about going to school. You've got to learn hard.
